In [ ]:
# Import required libraries
import os
from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Authenticate and set up Google Docs API
def authenticate_google_docs():
    """Authenticate and return the Google Docs API service."""
    try:
        auth.authenticate_user()
        creds, _ = default()
        service = build('docs', 'v1', credentials=creds)
        print("Google Docs API authenticated successfully.")
        return service
    except Exception as e:
        print(f"Error during authentication: {e}")
        return None

# Create a new Google Doc
def create_google_doc(service, title):
    """Create a new Google Doc and return its ID."""
    try:
        doc = service.documents().create(body={'title': title}).execute()
        print(f"Created Google Doc with ID: {doc.get('documentId')}")
        return doc.get('documentId')
    except HttpError as e:
        print(f"Error creating Google Doc: {e}")
        return None

# Convert markdown to Google Docs formatting
def format_markdown_to_docs(service, doc_id, markdown):
    """Convert markdown content to formatted Google Docs content."""
    try:
        requests = []
        lines = markdown.split('\n')

        for line in lines:
            if line.startswith('# '):  # Main title (Heading 1)
                requests.append({
                    'insertText': {
                        'location': {'index': 1},
                        'text': line[2:] + '\n'
                    }
                })
                requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': 1, 'endIndex': len(line) + 1},
                        'paragraphStyle': {'namedStyleType': 'HEADING_1'},
                        'fields': 'namedStyleType'
                    }
                })
            elif line.startswith('## '):  # Section headers (Heading 2)
                requests.append({
                    'insertText': {
                        'location': {'index': 1},
                        'text': line[3:] + '\n'
                    }
                })
                requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': 1, 'endIndex': len(line) + 1},
                        'paragraphStyle': {'namedStyleType': 'HEADING_2'},
                        'fields': 'namedStyleType'
                    }
                })
            elif line.startswith('### '):  # Sub-section headers (Heading 3)
                requests.append({
                    'insertText': {
                        'location': {'index': 1},
                        'text': line[4:] + '\n'
                    }
                })
                requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': 1, 'endIndex': len(line) + 1},
                        'paragraphStyle': {'namedStyleType': 'HEADING_3'},
                        'fields': 'namedStyleType'
                    }
                })
            elif line.startswith('- [ ]'):  # Checkboxes
                requests.append({
                    'insertText': {
                        'location': {'index': 1},
                        'text': line[6:] + '\n'
                    }
                })
                requests.append({
                    'createParagraphBullets': {
                        'range': {'startIndex': 1, 'endIndex': len(line) + 1},
                        'bulletPreset': 'BULLET_CHECKBOX'
                    }
                })
            elif line.startswith('* '):  # Bullet points
                requests.append({
                    'insertText': {
                        'location': {'index': 1},
                        'text': line[2:] + '\n'
                    }
                })
                requests.append({
                    'createParagraphBullets': {
                        'range': {'startIndex': 1, 'endIndex': len(line) + 1},
                        'bulletPreset': 'BULLET_DISC_CIRCLE_SQUARE'
                    }
                })
            elif '@' in line:  # Assignee mentions (bold)
                requests.append({
                    'insertText': {
                        'location': {'index': 1},
                        'text': line + '\n'
                    }
                })
                start_index = line.find('@')
                end_index = line.find(' ', start_index)
                if end_index == -1:
                    end_index = len(line)
                requests.append({
                    'updateTextStyle': {
                        'range': {'startIndex': 1 + start_index, 'endIndex': 1 + end_index},
                        'textStyle': {'bold': True},
                        'fields': 'bold'
                    }
                })
            else:  # Regular text
                requests.append({
                    'insertText': {
                        'location': {'index': 1},
                        'text': line + '\n'
                    }
                })

        # Apply all requests to the document
        service.documents().batchUpdate(
            documentId=doc_id,
            body={'requests': requests}
        ).execute()
        print("Markdown content formatted and added to Google Doc.")
    except HttpError as e:
        print(f"Error formatting Google Doc: {e}")

# Main function
def main():
    # Markdown meeting notes
    markdown = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

    # Authenticate and create Google Doc
    service = authenticate_google_docs()
    if service:
        doc_id = create_google_doc(service, "Product Team Sync - May 15, 2023")
        if doc_id:
            format_markdown_to_docs(service, doc_id, markdown)

# Run the script
if __name__ == "__main__":
    main()

Google Docs API authenticated successfully.
Created Google Doc with ID: 1gPFDytHDSX4ttHkaWqU_qfQlp5YmZxpwqLBWbEVo4Rk
Markdown content formatted and added to Google Doc.
